In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [1]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

prev_x, prev_y = None, None  

canvas = 255 * np.ones((480, 640, 3), dtype=np.uint8)

def distance(point1, point2):
    return math.sqrt((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2)

# if a finger is open based on the distance 
def is_finger_open(hand_landmarks, tip_id, base_id, dynamic_threshold):
    tip = hand_landmarks.landmark[tip_id]
    base = hand_landmarks.landmark[base_id]
    dist = distance(tip, base)
    
    # Use dynamic threshold for determining if a finger is open
    return dist > dynamic_threshold, dist  # Return distance for display

# Custom function for thumb
def is_thumb_open(hand_landmarks, tip_id, base_id, dynamic_threshold):
    tip = hand_landmarks.landmark[tip_id]
    base = hand_landmarks.landmark[base_id]
    dist = distance(tip, base)
    
    # Use dynamic threshold if thumb is open
    return dist > dynamic_threshold*2, dist  

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip the image 

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Dynamically compute the base distance (from middle finger base to wrist)
            dynamic_base_dist = distance(hand_landmarks.landmark[13], hand_landmarks.landmark[0])
            dynamic_threshold = dynamic_base_dist * 0.4  # You can tweak the multiplier as needed

            # index finger open 
            index_open, index_dist = is_finger_open(hand_landmarks, 8, 5, dynamic_threshold)

            # middle finger open
            middle_open, middle_dist = is_finger_open(hand_landmarks, 12, 9, dynamic_threshold)

            # ring finger open
            ring_open, ring_dist = is_finger_open(hand_landmarks, 16, 13, dynamic_threshold)

            # pinky open
            pinky_open, pinky_dist = is_finger_open(hand_landmarks, 20, 17, dynamic_threshold)

            # thumb open 
            thumb_open, thumb_dist = is_thumb_open(hand_landmarks, 4, 1, dynamic_threshold)

            # Drawing mode:
            if index_open and not ring_open and not thumb_open:
                index_tip = hand_landmarks.landmark[8]
                h, w, c = frame.shape
                cx, cy = int(index_tip.x * w), int(index_tip.y * h)

                # Start drawing
                cv2.circle(frame, (cx, cy), 15, (255, 0, 0), cv2.FILLED)

                if prev_x is not None and prev_y is not None:
                    cv2.line(canvas, (prev_x, prev_y), (cx, cy), (0, 0, 255), 5)
                prev_x, prev_y = cx, cy

            # Erasing mode: 
            elif ring_open and not index_open and not thumb_open:
                # Get ring finger tip coordinates
                ring_tip = hand_landmarks.landmark[16]
                h, w, c = frame.shape
                cx, cy = int(ring_tip.x * w), int(ring_tip.y * h)

                cv2.circle(canvas, (cx, cy), 20, (255, 255, 255), -1)
                cv2.circle(frame, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

            elif thumb_open and not index_open and not ring_open:
                # Clear the canvas
                canvas = 255 * np.ones((480, 640, 3), dtype=np.uint8)

            else:
                prev_x, prev_y = None, None

            h, w, c = frame.shape

            index_tip_coords = (int(hand_landmarks.landmark[8].x * w), int(hand_landmarks.landmark[8].y * h))
            cv2.putText(frame, f"Index: {'Open' if index_open else 'Closed'}", (index_tip_coords[0], index_tip_coords[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.putText(frame, f"Dist: {index_dist:.2f}", (index_tip_coords[0], index_tip_coords[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            middle_tip_coords = (int(hand_landmarks.landmark[12].x * w), int(hand_landmarks.landmark[12].y * h))
            cv2.putText(frame, f"Middle: {'Open' if middle_open else 'Closed'}", (middle_tip_coords[0], middle_tip_coords[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame, f"Dist: {middle_dist:.2f}", (middle_tip_coords[0], middle_tip_coords[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            ring_tip_coords = (int(hand_landmarks.landmark[16].x * w), int(hand_landmarks.landmark[16].y * h))
            cv2.putText(frame, f"Ring: {'Open' if ring_open else 'Closed'}", (ring_tip_coords[0], ring_tip_coords[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, f"Dist: {ring_dist:.2f}", (ring_tip_coords[0], ring_tip_coords[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            pinky_tip_coords = (int(hand_landmarks.landmark[20].x * w), int(hand_landmarks.landmark[20].y * h))
            cv2.putText(frame, f"Pinky: {'Open' if pinky_open else 'Closed'}", (pinky_tip_coords[0], pinky_tip_coords[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            cv2.putText(frame, f"Dist: {pinky_dist:.2f}", (pinky_tip_coords[0], pinky_tip_coords[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

            thumb_tip_coords = (int(hand_landmarks.landmark[4].x * w), int(hand_landmarks.landmark[4].y * h))
            cv2.putText(frame, f"Thumb: {'Open' if thumb_open else 'Closed'}", (thumb_tip_coords[0], thumb_tip_coords[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            cv2.putText(frame, f"Dist: {thumb_dist:.2f}", (thumb_tip_coords[0], thumb_tip_coords[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    cv2.imshow("Virtual Whiteboard", combined)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break  

cap.release()
cv2.destroyAllWindows()


NameError: name 'mp' is not defined

In [5]:
import requests
import sys
sys.path.append("E:\\Gowtham\\Lol")

from PIL import Image
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Function to perform OCR using Tesseract
def perform_ocr(api_key, image_path):
    with open(image_path, 'rb') as image_file:
        response = requests.post(
            'https://api.ocr.space/parse/image',
            files={'image': image_file},
            data={'apikey': api_key, 'OCREngine': 2}
        )
    result = response.json()
    return result['ParsedResults'][0]['ParsedText']

# Main execution
image_path = 'C:\\Users\\Gowtham\\Downloads\\halo aksay.png'
ocr_api_key = 'K88272180388957'

# Step 1: Perform OCR using Tesseract
recognized_text = perform_ocr(ocr_api_key,image_path)
print("Recognized Text:")
print(recognized_text)


Recognized Text:
Hello
Aksay anthan
